# Additional information for creating a cruiseplan 
Run the cell below to download required data and scripts

In [ ]:
import json 
import os 
import requests 

files = {
    "GLO-MFC_001_024_mask_bathy.nc":"https://surfdrive.surf.nl/files/index.php/s/AdbtlgP3LJv6tOn/download",
    "virtualship.py":"https://raw.githubusercontent.com/OceanParcels/Virtual_ship_classroom/main/virtualship.py"}

for filename, url in files.items():
  response = requests.get(url, allow_redirects=True)

  if response.status_code == 200:
      with open(filename, "wb") as f:
          f.write(response.content)
  else:
      print("Failed to download", url)
print('Download ready')

### Check your JSON file:
After you filled in your cruise proposal template and your JSON file, please run the following cell to check the input and correct your JSON file if necessary: 

In [ ]:
from virtualship import VirtualShipConfiguration
config = VirtualShipConfiguration('student_input.json') 

### Ship cruising speed

The cruising speed of the research vessel (i.e. the max speed between two locations without performing operations) is 10 knots. A knot is a unit of speed, equal to one nautical mile per hour. Something traveling at one knot is going about 1.151 land miles per hour. 10 knots = 18.52 kilometers per hour.  

### Cruise map

A possible way to create a list of coordinates and/or map of your cruise is to use an interactive leafmap.  

In [4]:
import leafmap 
from ipyleaflet import Rectangle 
 
m = leafmap.leafmap.Map(center=(0, -100), zoom=5) 
# Draw a rectange to indicate region where data will be collected 
m.add_layer(Rectangle(bounds=((64, -43), (59, -23)))) 
# Add a point at port of departure: Puntarenas, Costa Rica 
m.add_marker([9.97625, -84.83836]) 
m 

Map(center=[0, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

You can use the feature “Draw a polyline” (5th symbol from the top) to assemble a list of coordinates. Each point you click will show up as a white square and can be given as output. The blue lines can be used as transects. Double click the last point to finish the line (or click Finish). Create your output with: 

In [2]:
line = m.user_rois 
coords = line.get('features')[0].get('geometry').get('coordinates') 
print(coords) 

[[-23.071289, 63.743631], [-24.873047, 63.352129], [-26.630859, 62.915233], [-28.78418, 62.390369], [-30.322266, 61.918271], [-32.124023, 61.501734], [-34.145508, 60.930432], [-36.166992, 60.500525], [-38.408203, 59.95501], [-40.78125, 59.534318], [-42.539062, 59.800634]]


### Total cruise time 

To estimate the cruise time, you’ll need to divide the travelled distance by the cruise speed. To calculate the distance between 2 geographic coordinates you can use the haversine function (https://community.esri.com/t5/coordinate-reference-systems-blog/distance-on-a-sphere-the-haversine-formula/ba-p/902128) or an existing python package, for example geopy (https://geopy.readthedocs.io/en/stable/#module-geopy.distance) or haversine (https://pypi.org/project/haversine/) 

In [ ]:
from geopy import distance
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
print(distance.great_circle(newport_ri, cleveland_oh).nautical)

### Depth sampling 

A possible way to investigate the depth where you’ll be sampling is to use the bathymetry data that the virtual ship will also use.

An example for plotting and querying is given below.

In [ ]:
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs 
import xarray as xr 
 
# Load the bathymetry data using xarray 
data = xr.open_dataset('YOUR_OWN_DATA_PATH/GLO-MFC_001_024_mask_bathy.nc') 
# Create a bathymetry figure and axis with Cartopy projection 
fig, ax = plt.subplots(figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()}) 
data.deptho.plot(ax=ax, cmap='viridis') 
# Specify extent, add gridlines and coastlines, show plot 
ax.set_extent((130,160,-70,-40), crs=ccrs.PlateCarree()) 
ax.gridlines(draw_labels=True) 
ax.coastlines() 
plt.show() 
# Query and print the bathymetry data at the specified location 
station_depth = data.deptho.sel(latitude=-50, longitude=150, method='nearest') 
print(station_depth.values) 

## Measurement options 

### Underway data 

Underway data collection, also known as underway sampling, refers to the process of collecting oceanographic and environmental data while a research vessel is in motion. This method allows scientists to continuously gather data on various parameters such as sea surface temperature and salinity. The water inlet is located onthe hull of the ship at approximately 2 meters under the surface.  

You can collect underway data during your entire cruise and may assume it doesn’t take any time to start or stop taking measurements. 

### CTD 

A Conductivity, Temperature, and Depth (CTD) sensor is an instrument used to measure the physical properties of seawater in oceanographic and environmental research. It typically consists of sensors for conductivity, temperature, and pressure (which is used to derive depth). 

Throughout a CTD deployment, the research vessels needs to stay in the same location. To calculate the time needed for a CTD deployment, take the following information into account: 

Add time to lower the vessels speed to zero and set the vessel to the appropriate direction against swell, current, wind, etc = 10 minutes 
Add time to deploy the CTD, and to recover = 10 minutes (2*5 min) 
Add time for the way down and up, usually at 1 m/s (winch capacity) = twice your water depth (in seconds) 

https://youtu.be/7N2UsPDczTw?feature=shared 

<iframe width="560" height="315" src="https://www.youtube.com/embed/7N2UsPDczTw?si=Qj2e74UzIJe9OSDF&amp;controls=0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>


### ADCP  

An Acoustic Doppler Current Profiler (ADCP) is an instrument used to measure the speed and direction of ocean currents at various depths. It operates based on the principle of Doppler shift, where sound waves emitted by the instrument are reflected off moving particles in the water, such as plankton or sediment, and the frequency shift of the returning waves is used to calculate the velocity of the water. 

You can deploy the ADCP on transects and choose between a shallow profiler capable of providing information to a depth of 150 m every 4 meters (the 300kHz seaSeven), or a long-range profiler capable of providing about 1000m of range every 24 meters (the 38kHz Ocean Observer). You may assume it doesn’t take any time to start or end ADCP measurements.  

https://youtu.be/mDyoZagght8?feature=shared&t=38 

<iframe width="560" height="315" src="https://www.youtube.com/embed/mDyoZagght8?si=O_IeZaqajitB8f-Y&amp;controls=0&amp;start=38" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

### Drifters  

A surface drifter is an oceanographic instrument used to study surface currents and ocean circulation patterns. These devices are designed to drift passively with the surface currents while transmitting data on their position and temperature. 

You may assume it doesn’t take any time to deploy drifters. 

https://youtu.be/3m_CxEE8Bhs?feature=shared&t=15 

<iframe width="560" height="315" src="https://www.youtube.com/embed/3m_CxEE8Bhs?si=1ysBR9uAkP7U20rk&amp;controls=0&amp;start=15" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

### Argo floats 

Argo floats are self-contained, battery-powered instruments equipped with sensors to measure temperature and salinity profiles from the surface down to depths of up to 2000 meters or more. These sensors provide high-resolution vertical profiles of ocean properties, allowing researchers to study ocean variability and climate change. 

The floats periodically descend to a predetermined depth, typically around 1000 meters, and then ascend to the surface, measuring temperature and salinity profiles along the way. In tutorial 2, you learn how to control the dive depth and duration.  

You may assume it doesn’t take any time to deploy Argo floats. 

https://youtu.be/IgcYQML5se4?feature=shared 

<iframe width="560" height="315" src="https://www.youtube.com/embed/IgcYQML5se4?si=TTcEfb8sR5Z9BJIo&amp;controls=0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## Additional information 

 

Things not to worry about on your virtual cruise, but which are important when planning an actual cruise: 

 

### Planning 

Preparing for a seagoing expedition involves many months (often years) of planning. There are many forms and documents to prepare; there are many criteria to be met and logistical issues to sort out; there are even issues of personal preparation. The most successful research cruises are typically the ones that are planned early, with an awareness of both the big picture and the fine details. Research ships are often fully booked over a year in advance. Collaborative planning ensures that cruises in neighboring regions are scheduled together to minimize transit time. To gain insights into the schedules of various European research ships, you can e.g. refer to https://www.marinefacilitiesplanning.com/programme. 

 

### Diplomatic clearance 

If the cruise includes data collection and/or sampling in other countries exclusive economic zones (EEZ) the cruise leader must seek permission through diplomatic channels ref UNCLOS (United Nations Convention of the Laws Of the Seas) Part XIII Marine Scientific Research. Normal processing time is at least 3 to 6 months. This is especially important when sampling on or below the seafloor, trawling, and collecting high resolution bathymetric data. 

  

### Polar code  

Research vessels have different polar capabilities and it therefore varies in which areas they can operate at different times of the year, depending on ice conditions and minimum allowed air temperature. 

 

### Crew shifts 

Many research vessels operate on a watch system, where crew members are divided into teams (watches) that take turns being responsible for the vessel's operations. This ensures that the vessel is staffed 24 hours a day, allowing for continuous data collection and monitoring. Traditional watch schedules include the “12-on, 12-off", “4-on, 8-off" or "6-on, 6-off" rotation, where crew members work for a set number of hours (e.g., 4 or 6 hours) and then have a set amount of time off duty. This rotation continues throughout the day and night. Depending on the requirements of the research mission, modified watch schedules may be implemented. 

 

### Logistics (transportation, customs, intermediate storage) 

All equipment required at sea needs to be transferred to the port of departure before leaving. Consider these issues on how to prepare and pack your equipment in the office: 

Create a comprehensive checklist of all the equipment you'll need for your research. This includes scientific instruments, sampling tools, safety gear, and any other materials essential to your work.  
Clean and calibrate your instruments as needed. Ensure that sensors are functioning properly, batteries are charged, and any necessary software is updated. Make note of any items that may require special handling or storage. 
Organize your equipment in a systematic manner. Group items together based on their function or use. Clearly label each item with its name, purpose, and any specific instructions for handling or setup. 
Fit your equipment in the 20-foot container. The container will be sent to your port of departure ahead of time with a cargo boat, so make sure you are packed in time for this transfer. Remember there are no shops at sea, so think carefully and plan ahead. 
You are responsible for ensuring the equipment is loaded or unloaded from the vessel. You should plan to have a responsible person at the port stop for loading or unloading your equipment. Please do not assume the crew will take care of this for you. 

![Equipment preparation NIOZ](https://www.nioz.nl/application/files/9116/7500/3457/2023-01-16-packing.jpg) 

![Equipment loading](https://www.nioz.nl/application/files/7416/7810/2265/2023-03-06-container-shifting.jpg) 